# Testing multiple models
in this notebook, multiple models will be made with the png's of the Lactic acid experiment
(mostly bad results because of not enough data) 

In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
from random import sample

from transform_functions import get_data_rf, get_data


from sklearn import linear_model

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn import linear_model, datasets
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree



## Model 1 
simple RF classifier, no balance found/not found, train all data at once 

In [2]:
PATH = "/media/sf_SF/Stage2021/test_ML/" #test directory

#get data
X_train,y_train = get_data(PATH+"ML_train/")
X_test, y_test = get_data(PATH+"ML_test/")

In [4]:
rf_classifier = RandomForestClassifier(max_depth = 2, random_state = 0)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))
print("\nconfusion matrix")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\naccuracy")
print(accuracy_score(y_test, y_pred) * 100) 

/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.84      1.00      0.91        42

    accuracy                           0.84        50
   macro avg       0.42      0.50      0.46        50
weighted avg       0.71      0.84      0.77        50


confusion matrix
[[ 0  8]
 [ 0 42]]

accuracy
84.0


/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 2
Png's found and not found are balanced in the train (2/3) and test set (1/3).
The train data is split up in multiple sets, the model grows with warm start.


In [10]:
#select project map where found + nf directories are
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

founds = os.listdir(PROJECT+"found/")
length_fo = len(founds)

not_founds = os.listdir(PROJECT+"not_found/")
length_nf = len(not_founds)

print(length_fo)
print(length_nf)

#divide data
#1/3 of founds = test data, 2/3 of founds = train data
test_fo = (sample(founds,round(length_fo/3)))
train_fo = list(set(founds) - set(test_fo))

#1/3 of not founds = test data, 2/3 of not founds = train data
test_nf = (sample(not_founds,round(length_nf/3)))
train_nf = list(set(not_founds) - set(test_nf))

#divide into two lists
all_test = test_fo + test_nf
all_train = train_fo + train_nf

print("nf_test : ", len(test_nf), "f_test", len(test_fo))
print("nf_train : ", len(test_nf), "f_train", len(train_fo))

print("all_test : ", len(all_test))
print("all_train : ", len(all_train))

#select test data 
sel_test = (sample(all_test, 10))
X_test, y_test = get_data_rf(PROJECT, sel_test)
print(X_test)
print(y_test)

#make classifier
cls = RandomForestClassifier(n_estimators=1000, warm_start=True, max_features=2)

#make loop to create different train data sets (let the model grow with warm start, goes faster for big data)
for j in range(3): #now 2 times loop, bcs not enough data 
    X_train=[]
    y_train=[]
    #make subgroup to train the data 
    sel_train_fo = sample(train_fo,4)
    sel_train_nf = sample(train_nf,4)
    # print(sel_train_fo)
    # print(sel_train_nf)
    sel_train = sel_train_fo + sel_train_nf
    train_nf = list(set(train_nf) - set(sel_train_nf))
    train_fo = list(set(train_fo) - set(sel_train_fo))
    X_train,y_train = get_data_rf(PROJECT, sel_train)
    print(X_train)
    print(y_train)   
    cls.n_estimators += 10 
    cls.fit(X_train, y_train)
    
    print("-"*10)
    #Hoe belangrijk is elke feature voor het model (0.22 en 0.15 zullen dan degene zijn die gebruikt worden aangezien je slecht 2 features gebruikt hier)
    print(cls.feature_importances_)
    #Vb van 1 boom
    #plt.figure(figsize=(80,80))
    #_ = tree.plot_tree(cls.estimators_[0], filled=True)


124
26
nf_test :  9 f_test 41
nf_train :  9 f_train 83
all_test :  50
all_train :  100
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[1 1 1 1 1 1 1 0 1 1]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[1 1 1 1 0 0 0 0]
----------
[0. 0. 0. ... 0. 0. 0.]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[1 1 1 1 0 0 0 0]
----------
[0. 0. 0. ... 0. 0. 0.]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[1 1 1 1 0 0 0 0]

*feature importance is steeds nul bij mij?*

In [11]:
# Test Random Forest Classifier
y_pred = cls.predict(X_test)
print(classification_report(y_test, y_pred))
print("\nconfusion matrix")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\naccuracy")
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       1.00      0.67      0.80         9

    accuracy                           0.70        10
   macro avg       0.62      0.83      0.60        10
weighted avg       0.93      0.70      0.76        10


confusion matrix
[[1 0]
 [3 6]]

accuracy
70.0


*waarom een accuracy van slechts 40 als het eerste model dubbel zoveel is?*

## Model 3
the train data is not split up here 
(max_features bepaalt hoeveel features gebruikt mogen worden per split per boom. Zo bekom je de "randomness" in de random forest trees)

In [12]:
#select project map where found + nf directories are
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

founds = os.listdir(PROJECT+"found/")
length_fo = len(founds)

not_founds = os.listdir(PROJECT+"not_found/")
length_nf = len(not_founds)

print(length_fo)
print(length_nf)

#divide data
#1/3 of founds = test data, 2/3 of founds = train data
test_fo = (sample(founds,round(length_fo/3)))
train_fo = list(set(founds) - set(test_fo))

#1/3 of not founds = test data, 2/3 of not founds = train data
test_nf = (sample(not_founds,round(length_nf/3)))
train_nf = list(set(not_founds) - set(test_nf))

#divide into two lists
all_test = test_fo + test_nf
all_train = train_fo + train_nf

print("nf_test : ", len(test_nf), "f_test", len(test_fo))
print("nf_train : ", len(test_nf), "f_train", len(train_fo))

print("all_test : ", len(all_test))
print("all_train : ", len(all_train))

#get X_train, y_train... 
X_test, y_test = get_data_rf(PROJECT, all_test)
X_train,y_train = get_data_rf(PROJECT, all_train)
print(X_test)
print(y_test)

#make classifier
cls2 = RandomForestClassifier(n_estimators=1000, max_features=2)
cls2.fit(X_train, y_train)


124
26
nf_test :  9 f_test 41
nf_train :  9 f_train 83
all_test :  50
all_train :  100
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0]


RandomForestClassifier(max_features=2, n_estimators=1000)

In [13]:
# Test Random Forest Classifier
y_pred = cls2.predict(X_test)
print("\nprediction")
print(classification_report(y_test, y_pred))
print("\nconfusion matrix")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\naccuracy")
print(accuracy_score(y_test, y_pred) * 100) 


prediction
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.82      1.00      0.90        41

    accuracy                           0.82        50
   macro avg       0.41      0.50      0.45        50
weighted avg       0.67      0.82      0.74        50


confusion matrix
[[ 0  9]
 [ 0 41]]

accuracy
82.0


/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gradient Boosting 

In [15]:
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

X_train,y_train = get_data(PROJECT+"ML_train/")
X_test, y_test = get_data(PROJECT+"ML_test/")

clf = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("\nconfusion matrix")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\naccuracy")
print(accuracy_score(y_test, y_pred) * 100) 

              precision    recall  f1-score   support

           0       0.71      0.62      0.67         8
           1       0.93      0.95      0.94        42

    accuracy                           0.90        50
   macro avg       0.82      0.79      0.80        50
weighted avg       0.90      0.90      0.90        50


confusion matrix
[[ 5  3]
 [ 2 40]]

accuracy
90.0


### Bagging
Kan je boom verbeteren tijdens trainen. Hier moet je dan beetje spelen met de complexity om eventueel betere resultaten te krijgen

In [17]:
# Bagging met logistic regression
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

X_train,y_train = get_data(PROJECT+"ML_train/")
X_test, y_test = get_data(PROJECT+"ML_test/")

number_of_estimators = 100
complexity = 10
cart = LogisticRegression(C=complexity,solver='liblinear')


lregbagging = BaggingClassifier(base_estimator=cart, n_estimators=number_of_estimators)
lregbagging.fit(X_train,y_train)
y_pred = lregbagging.predict(X_test)

print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.91      0.98      0.94        42

    accuracy                           0.90        50
   macro avg       0.86      0.74      0.78        50
weighted avg       0.89      0.90      0.89        50

[[ 4  4]
 [ 1 41]]
90.0


### Boosting
Andere manier naast bagging die ook rekening houd met resultaten van vorige training, hier spelen met learning rate

In [18]:
# Adaboost
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

X_train,y_train = get_data(PROJECT+"ML_train/")
X_test, y_test = get_data(PROJECT+"ML_test/")

clf_adaboost = AdaBoostClassifier(n_estimators=150,learning_rate=0.9)
clf_adaboost.fit(X_train,y_train)

y_pred = clf_adaboost.predict(X_test)

print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 


              precision    recall  f1-score   support

           0       0.86      0.75      0.80         8
           1       0.95      0.98      0.96        42

    accuracy                           0.94        50
   macro avg       0.91      0.86      0.88        50
weighted avg       0.94      0.94      0.94        50

[[ 6  2]
 [ 1 41]]
94.0


### Logistic regression

Geen boom, gewoon simpele logistic regression. Er onder nog een paar andere boosting technieken. XDGboost wordt momenteel redelijk vaak gebruikt

In [19]:
# Adaboost met logistic regression classifier
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

X_train,y_train = get_data(PROJECT+"ML_train/")
X_test, y_test = get_data(PROJECT+"ML_test/")

cart = LogisticRegression(C=complexity,solver='liblinear')
logreg_adaboost = AdaBoostClassifier(base_estimator=cart,n_estimators=150,learning_rate=0.9) 
logreg_adaboost.fit(X_train,y_train)

y_pred = logreg_adaboost.predict(X_test)

print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 



              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.91      0.98      0.94        42

    accuracy                           0.90        50
   macro avg       0.86      0.74      0.78        50
weighted avg       0.89      0.90      0.89        50

[[ 4  4]
 [ 1 41]]
90.0


In [16]:
# Gradient boosting
PROJECT = "/media/sf_SF/Stage2021/test_ML/"

X_train,y_train = get_data(PROJECT+"ML_train/")
X_test, y_test = get_data(PROJECT+"ML_test/")

clf_gradientboost = GradientBoostingClassifier(n_estimators=150,learning_rate=0.8)
clf_gradientboost.fit(X_train,y_train)


y_pred = clf_gradientboost.predict(X_test)
print(classification_report(y_test, y_pred))
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 



              precision    recall  f1-score   support

           0       0.75      0.38      0.50         8
           1       0.89      0.98      0.93        42

    accuracy                           0.88        50
   macro avg       0.82      0.68      0.72        50
weighted avg       0.87      0.88      0.86        50

[[ 3  5]
 [ 1 41]]
88.0
